<a href="https://colab.research.google.com/github/ajisakti554/Big-Data/blob/main/Pyspark_tubes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TUGAS BESAR BIG DATA (preprocessing )**

Nama kelompok :

1.   Aji Sakti Saputra
2.   Fadhli Jahfal
3.   Rafid Farhan Zai
4.   Yusuf Sohibul


#1.  Instal Java dan PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

# 2. Set Environment Variabel

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.11/dist-packages/pyspark"

# 3. Inisialisasi SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Colab PySpark").getOrCreate()

In [ ]:
# Upload file dari komputer lokal
from google.colab import files
uploaded = files.upload()


Saving lobster_vessels_dataset.csv to lobster_vessels_dataset.csv
Saving lobster_quality_inspections_dataset.csv to lobster_quality_inspections_dataset.csv
Saving lobster_market_sales_dataset.csv to lobster_market_sales_dataset.csv
Saving lobster_fishing_grounds_dataset.csv to lobster_fishing_grounds_dataset.csv
Saving lobster_fishers_dataset.csv to lobster_fishers_dataset.csv
Saving lobster_dataset.csv to lobster_dataset.csv


# Input dataset pertama yaitu dataset lobster_dataset

In [ ]:
# Untuk CSV
df_lobster = spark.read.option("header", "true").option("inferSchema", "true").csv("lobster_dataset.csv")
df_lobster.show()

+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|lobster_id|         species|catch_date|length_cm|weight_grams|carapace_length_cm|  grade|shell_condition|catch_method|fisher_id|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|
+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|  LOB00001|American Lobster| 10/5/2023|    15.75|       557.5|             10.23|Grade A|     Hard Shell|Hand Picking|   FSH005|   VSL002|            FG003|              37.03|    true|             true|       false|
|  LOB00002|European Lobster|  8/9/2024|    28.48|       561.6|             16.69|Grade B|    Paper Shell|    Trawling|   FSH036

**Karena dataset ke 1 memiliki data yang null dan duplikasi, maka cleaning terlebih dahulu**

In [ ]:
# 1. Cek missing values
from pyspark.sql.functions import col, sum

missing_values = df_lobster.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_lobster.columns])
missing_values.show()

+----------+-------+----------+---------+------------+------------------+-----+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|lobster_id|species|catch_date|length_cm|weight_grams|carapace_length_cm|grade|shell_condition|catch_method|fisher_id|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|
+----------+-------+----------+---------+------------+------------------+-----+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|         0|      0|         0|        2|           1|                 0|    0|              1|           0|        0|        0|                0|                  0|       0|                0|           0|
+----------+-------+----------+---------+------------+------------------+-----+---------------+------------+---------+---------+-----------------+-------------------+------

In [ ]:
# 2. Isi missing value di kolom length_cm dengan nilai median
from pyspark.sql.functions import median

median_height =df_lobster.select(median("length_cm")).first()[0]
df_lobster = df_lobster.na.fill(median_height, subset=["length_cm"])

df_lobster.show()

+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|lobster_id|         species|catch_date|length_cm|weight_grams|carapace_length_cm|  grade|shell_condition|catch_method|fisher_id|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|
+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|  LOB00001|American Lobster| 10/5/2023|    15.75|       557.5|             10.23|Grade A|     Hard Shell|Hand Picking|   FSH005|   VSL002|            FG003|              37.03|    true|             true|       false|
|  LOB00002|European Lobster|  8/9/2024|    28.48|       561.6|             16.69|Grade B|    Paper Shell|    Trawling|   FSH036

In [ ]:
# 2. Isi missing value di kolom  weight_gram dengan nilai median
from pyspark.sql.functions import median

median_weight =df_lobster.select(median("weight_grams")).first()[0]
df_lobster = df_lobster.na.fill(median_weight, subset=["weight_grams"])

df_lobster.show()

+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|lobster_id|         species|catch_date|length_cm|weight_grams|carapace_length_cm|  grade|shell_condition|catch_method|fisher_id|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|
+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|  LOB00001|American Lobster| 10/5/2023|    15.75|       557.5|             10.23|Grade A|     Hard Shell|Hand Picking|   FSH005|   VSL002|            FG003|              37.03|    true|             true|       false|
|  LOB00002|European Lobster|  8/9/2024|    28.48|       561.6|             16.69|Grade B|    Paper Shell|    Trawling|   FSH036

In [ ]:
#isi missing value di kolom shell_condition dengan nilai ffill
from pyspark.sql.functions import col

# ✅ SOLUSI 1: Menggunakan fillna() dengan nilai default
df_lobster = df_lobster.withColumn("shell_condition", col("shell_condition").cast("string"))
df_lobster = df_lobster.fillna("Unknown", subset=["shell_condition"])
df_lobster.show()



+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|lobster_id|         species|catch_date|length_cm|weight_grams|carapace_length_cm|  grade|shell_condition|catch_method|fisher_id|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|
+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|  LOB00001|American Lobster| 10/5/2023|    15.75|       557.5|             10.23|Grade A|     Hard Shell|Hand Picking|   FSH005|   VSL002|            FG003|              37.03|    true|             true|       false|
|  LOB00002|European Lobster|  8/9/2024|    28.48|       561.6|             16.69|Grade B|    Paper Shell|    Trawling|   FSH036

In [ ]:
# 1. Cek missing values kembali untuk memastikan dataset sudah clean
from pyspark.sql.functions import col, sum

missing_values = df_lobster.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_lobster.columns])
missing_values.show()

+----------+-------+----------+---------+------------+------------------+-----+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|lobster_id|species|catch_date|length_cm|weight_grams|carapace_length_cm|grade|shell_condition|catch_method|fisher_id|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|
+----------+-------+----------+---------+------------+------------------+-----+---------------+------------+---------+---------+-----------------+-------------------+--------+-----------------+------------+
|         0|      0|         0|        0|           0|                 0|    0|              0|           0|        0|        0|                0|                  0|       0|                0|           0|
+----------+-------+----------+---------+------------+------------------+-----+---------------+------------+---------+---------+-----------------+-------------------+------

# Dataset kedua

In [ ]:
# Untuk CSV
df_lobster_fisher = df_lobster_fisher = spark.read.option("header", "true") \
                               .option("inferSchema", "true") \
                               .option("multiLine", "true") \
                               .option("quote", '"') \
                               .option("escape", '"') \
                               .option("mode", "PERMISSIVE") \
                               .csv("lobster_fishers_dataset.csv")
df_lobster_fisher.show()

+---------+-----------------+--------------+--------------------+--------------------+------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+
|fisher_id|      fisher_name|license_number|             address|                city|       state|               phone|               email|experience_years|license_expiry|vessel_owner|crew_size|annual_quota_kg|total_catch_this_year|
+---------+-----------------+--------------+--------------------+--------------------+------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+
|   FSH001|  Courtney Harris|      LIC31125|624 Samantha Aven...|          Port Robin|   Minnesota|     +1-416-342-6015|hamiltoncassandra...|               1|    2027-01-10|        true|        8|           1329|                 2819|
|   FSH002|       Jade Banks|      LIC17002|Unit 4378 Box 75

# Dataset ke 3

In [ ]:
df_lobster_fishing_grounds = spark.read.option("header", "true").option("inferSchema", "true").csv("lobster_fishing_grounds_dataset.csv")
df_lobster_fishing_grounds.show()

+-----------------+--------------------+---------+----------+------------------+-----------------+---------------------+------------------------+----------------+--------------+----------------------+-----------+----------------+--------------------+
|fishing_ground_id|         ground_name| latitude| longitude|water_depth_meters|       water_type|average_temperature_c|lobster_density_per_sqkm|seasonal_closure|protected_area|distance_from_shore_km|bottom_type|current_strength|accessibility_rating|
+-----------------+--------------------+---------+----------+------------------+-----------------+---------------------+------------------------+----------------+--------------+----------------------+-----------+----------------+--------------------+
|            FG001|     Bobbyland Ridge|41.093432|-74.396275|                53|       Deep Water|                  8.6|                     285|           false|         false|                  31.6|      Rocky|            Weak|                  

# Dataset ke 4

In [ ]:
df_lobster_market_sales_dataset=spark.read.option("header", "true").option("inferSchema", "true").csv("lobster_market_sales_dataset.csv")
df_lobster_market_sales_dataset.show()

+---------+----------+----------+-------------------+---------------+----------------+------------+--------------+------------------+-------------+--------------------+---------------+------------------+-------------------+
|  sale_id|lobster_id| sale_date|         buyer_name|   sale_channel|quantity_sold_kg|price_per_kg|transport_cost|commission_percent|payment_terms|   delivery_location|customer_rating|  total_sale_value|         net_profit|
+---------+----------+----------+-------------------+---------------+----------------+------------+--------------+------------------+-------------+--------------------+---------------+------------------+-------------------+
|SALE00001|  LOB08396|2025-04-16|   Seafood Market A|        Auction|            1.22|       40.31|         20.78|              12.9|       Net 30|          Port Shawn|              5|49.178200000000004| 22.054212200000002|
|SALE00002|  LOB07364|2025-02-27| Restaurant Chain B|    Direct Sale|             1.2|       51.76|     

# Dataset ke 5

In [ ]:
lobster_quality_inspections=df_lobster_quality_inspections=spark.read.option("header", "true").option("inferSchema", "true").csv("lobster_quality_inspections_dataset.csv")
lobster_quality_inspections.show()

+-------------+----------+---------------+--------------+-----------------+---------------+---------------+--------------+--------------+------------------+----------------+---------------+------------------+--------------------+
|inspection_id|lobster_id|inspection_date|inspector_name|visual_inspection|size_compliance|shell_integrity|  meat_quality|parasite_check|contamination_test|  overall_result|inspection_cost|certificate_issued|               notes|
+-------------+----------+---------------+--------------+-----------------+---------------+---------------+--------------+--------------+------------------+----------------+---------------+------------------+--------------------+
|     INS00001|  LOB09664|     2025-01-20|  Sarah Wilson|  Retest Required|          false|           Poor|Below Standard|   Significant|          Positive|Conditional Pass|           19.1|             false|Somebody compare ...|
|     INS00002|  LOB00866|     2025-01-05|  Sarah Wilson|  Retest Required|     

# Dataset ke 6

In [ ]:
lobster_vessels_dataset=spark.read.option("header", "true").option("inferSchema", "true").csv("lobster_vessels_dataset.csv")
lobster_vessels_dataset.show()

+---------+--------------------+---------------+-----------+---------+-------------+-----------------+---------------------+-------------+----------+-----------------+-------------------+---------------+------------+--------------+
|vessel_id|         vessel_name|    vessel_type|length_feet|beam_feet|hull_material|engine_horsepower|fuel_capacity_gallons|crew_capacity|built_year|        home_port|registration_number|insurance_value|gps_equipped|sonar_equipped|
+---------+--------------------+---------------+-----------+---------+-------------+-----------------+---------------------+-------------+----------+-----------------+-------------------+---------------+------------+--------------+
|   VSL001|Campbell, Young a...|Offshore Vessel|         56|       25|   Fiberglass|             1259|                 1687|           12|      2013|    Lake Victoria|            REG5146|        1445654|       false|          true|
|   VSL002|Hamilton-Osborne ...|Offshore Vessel|        112|       19|  

**Karena sebelumnya di pandas hanya dataset ke 1 yang memiliki dataset yang data kotor (null,duplikasi), jdi dapat dipastikan semua dataset siap di analisis**

# INNER JOIN DATASET 1 DAN 2

In [ ]:
# Inner Join berdasarkan kolom 'fisher_id'
joined_df = df_lobster.join(df_lobster_fisher, on="fisher_id", how="inner")

# Tampilkan hasil
joined_df.show()


+---------+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+-----------------+-------------------+--------+-----------------+------------+-----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+
|fisher_id|lobster_id|         species|catch_date|length_cm|weight_grams|carapace_length_cm|  grade|shell_condition|catch_method|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|      fisher_name|license_number|             address|                city|         state|               phone|               email|experience_years|license_expiry|vessel_owner|crew_size|annual_quota_kg|total_catch_this_year|
+---------+----------+----------------+----------+---------+------------+------------------+-------+--------------

membuat kolom baru

In [ ]:
from pyspark.sql.functions import to_date, year

# Ubah 'catch_date' ke format tanggal
joined_df = joined_df.withColumn("catch_date", to_date("catch_date", "M/d/yyyy"))

# Ambil tahun dan simpan di kolom baru 'catch_year'
joined_df = joined_df.withColumn("catch_year", year("catch_date"))


In [ ]:
joined_df.show()

+---------+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+-----------------+-------------------+--------+-----------------+------------+-----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+----------+
|fisher_id|lobster_id|         species|catch_date|length_cm|weight_grams|carapace_length_cm|  grade|shell_condition|catch_method|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|      fisher_name|license_number|             address|                city|         state|               phone|               email|experience_years|license_expiry|vessel_owner|crew_size|annual_quota_kg|total_catch_this_year|catch_year|
+---------+----------+----------------+----------+---------+------------+------------------+

In [ ]:
# 1. Cek missing values
from pyspark.sql.functions import col, sum

missing_values = joined_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in joined_df.columns])
missing_values.show()

+---------+----------+-------+----------+---------+------------+------------------+-----+---------------+------------+---------+-----------------+-------------------+--------+-----------------+------------+-----------+--------------+-------+----+-----+-----+-----+----------------+--------------+------------+---------+---------------+---------------------+----------+
|fisher_id|lobster_id|species|catch_date|length_cm|weight_grams|carapace_length_cm|grade|shell_condition|catch_method|vessel_id|fishing_ground_id|market_price_per_kg|is_alive|inspection_passed|export_ready|fisher_name|license_number|address|city|state|phone|email|experience_years|license_expiry|vessel_owner|crew_size|annual_quota_kg|total_catch_this_year|catch_year|
+---------+----------+-------+----------+---------+------------+------------------+-----+---------------+------------+---------+-----------------+-------------------+--------+-----------------+------------+-----------+--------------+-------+----+-----+-----+----

**1.menganalisis total tangkapan lobster di setiap tahunnya berdasarkan species**

In [ ]:
from pyspark.sql.functions import sum

tangkapan_lobster_sum = joined_df.groupBy("catch_year", "species") \
                                  .agg(sum("total_catch_this_year").alias("total_catch")) \
                                  .orderBy("total_catch", ascending=False)
tangkapan_lobster_sum.show()


+----------+----------------+-----------+
|catch_year|         species|total_catch|
+----------+----------------+-----------+
|      2024| Slipper Lobster|    2466671|
|      2024|European Lobster|    2440580|
|      2024|    Rock Lobster|    2388588|
|      2024|American Lobster|    2319937|
|      2024|   Spiny Lobster|    2288114|
|      2023|American Lobster|    1431251|
|      2023|European Lobster|    1369341|
|      2023|    Rock Lobster|    1351377|
|      2023| Slipper Lobster|    1351032|
|      2023|   Spiny Lobster|    1337283|
|      2025| Slipper Lobster|    1091912|
|      2025|    Rock Lobster|    1056055|
|      2025|American Lobster|     979226|
|      2025|   Spiny Lobster|     964814|
|      2025|European Lobster|     958030|
+----------+----------------+-----------+



In [ ]:
from pyspark.sql.functions import sum

tangkapan_lobster_sum = joined_df.groupBy("species", "catch_method") \
                                  .agg(sum("total_catch_this_year").alias("total_catch")) \
                                  .orderBy("total_catch", ascending=False)
tangkapan_lobster_sum.show()


+----------------+------------+-----------+
|         species|catch_method|total_catch|
+----------------+------------+-----------+
| Slipper Lobster|        Trap|    1337880|
|European Lobster|      Diving|    1254156|
| Slipper Lobster|Hand Picking|    1252037|
|   Spiny Lobster|      Diving|    1244691|
|European Lobster|Hand Picking|    1241343|
|American Lobster|Hand Picking|    1217589|
|    Rock Lobster|      Diving|    1211054|
| Slipper Lobster|    Trawling|    1206281|
|    Rock Lobster|    Trawling|    1203318|
|    Rock Lobster|        Trap|    1197480|
|American Lobster|      Diving|    1194787|
|    Rock Lobster|Hand Picking|    1184168|
|European Lobster|        Trap|    1167454|
|American Lobster|    Trawling|    1162371|
|American Lobster|        Trap|    1155667|
|   Spiny Lobster|    Trawling|    1141958|
|   Spiny Lobster|        Trap|    1115866|
| Slipper Lobster|      Diving|    1113417|
|European Lobster|    Trawling|    1104998|
|   Spiny Lobster|Hand Picking| 

**3. menghitung banyaknya tangkapan lobster berdasarkan metode penangkapan berdasarkan tahun ketahun**

PySpark tidak memiliki fungsi pivot_table seperti Pandas,jadi kami mendapatkan hasil yang setara dengan menggunakan method .groupBy().pivot().agg().



In [ ]:
from pyspark.sql.functions import sum

pivot_table = joined_df.groupBy("catch_method") \
                       .pivot("catch_year") \
                       .agg(sum("total_catch_this_year"))
pivot_table.show()


+------------+-------+-------+-------+
|catch_method|   2023|   2024|   2025|
+------------+-------+-------+-------+
|        Trap|1768444|2940590|1265313|
|    Trawling|1712509|2886978|1219439|
|      Diving|1675087|3031563|1311455|
|Hand Picking|1684244|3044759|1253830|
+------------+-------+-------+-------+



# INNER JOIN DATASETS 1 DAN 3

In [ ]:
# 8. Lakukan inner join antara joined_df dan df_lobster_fishing_grounds
joined_df = joined_df.join(df_lobster_fishing_grounds, on=["fishing_ground_id"], how="inner")

joined_df.show()

+-----------------+---------+----------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+-------------------+--------+-----------------+------------+-----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+----------+--------------------+---------+----------+------------------+-----------------+---------------------+------------------------+----------------+--------------+----------------------+-----------+----------------+--------------------+
|fishing_ground_id|fisher_id|lobster_id|         species|catch_date|length_cm|weight_grams|carapace_length_cm|  grade|shell_condition|catch_method|vessel_id|market_price_per_kg|is_alive|inspection_passed|export_ready|      fisher_name|license_number|             address|                city|         state|        

**4. melihat jumlah penangkapan lobster berdasarkan species dan type air**

In [ ]:
from pyspark.sql.functions import sum

pivott = joined_df.groupBy("species") \
                   .pivot("water_type") \
                   .agg(sum("total_catch_this_year").alias("total_catch"))

pivott.show()


+----------------+-------+-----------------+----------+------------+------------+
|         species|Coastal|Continental Shelf|Deep Water|Rocky Bottom|Sandy Bottom|
+----------------+-------+-----------------+----------+------------+------------+
|American Lobster|1444083|          1001493|   1169907|      544417|      570514|
|    Rock Lobster|1531130|           917649|   1222436|      522909|      601896|
| Slipper Lobster|1536801|          1051827|   1148966|      619837|      552184|
|   Spiny Lobster|1477261|           885875|   1041231|      624079|      561765|
|European Lobster|1475510|          1012382|   1122411|      552274|      605374|
+----------------+-------+-----------------+----------+------------+------------+



**5. melihat apakah total penangkapan lobster dari tahun ke tahun berada dalam musimnya/tidak**

In [ ]:
print(joined_df.columns)


['fishing_ground_id', 'fisher_id', 'lobster_id', 'species', 'catch_date', 'length_cm', 'weight_grams', 'carapace_length_cm', 'grade', 'shell_condition', 'catch_method', 'vessel_id', 'market_price_per_kg', 'is_alive', 'inspection_passed', 'export_ready', 'fisher_name', 'license_number', 'address', 'city', 'state', 'phone', 'email', 'experience_years', 'license_expiry', 'vessel_owner', 'crew_size', 'annual_quota_kg', 'total_catch_this_year', 'catch_year', 'ground_name', 'latitude', 'longitude', 'water_depth_meters', 'water_type', 'average_temperature_c', 'lobster_density_per_sqkm', 'seasonal_closure', 'protected_area', 'distance_from_shore_km', 'bottom_type', 'current_strength', 'accessibility_rating']


In [ ]:
from pyspark.sql.functions import sum

musim = joined_df.groupBy("species", "seasonal_closure_clean") \
    .agg(sum("total_catch_this_year").alias("total_catch"))

musim.show()


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `seasonal_closure_clean` cannot be resolved. Did you mean one of the following? [`seasonal_closure`, `sonar_equipped`, `total_sale_value`, `visual_inspection`, `annual_quota_kg`].;
'Aggregate [species#18, 'seasonal_closure_clean], [species#18, 'seasonal_closure_clean, sum(total_catch_this_year#906) AS total_catch#5567L]
+- Project [vessel_id#27, lobster_id#17, fishing_ground_id#28, fisher_id#26, species#18, catch_date#1644, length_cm#355, weight_grams#484, carapace_length_cm#22, grade#23, shell_condition#615, catch_method#25, market_price_per_kg#29, is_alive#30, inspection_passed#31, export_ready#32, fisher_name#894, license_number#895, address#896, city#897, state#898, phone#899, email#900, experience_years#901, ... 60 more fields]
   +- Join Inner, (vessel_id#27 = vessel_id#1361)
      :- Project [lobster_id#17, fishing_ground_id#28, fisher_id#26, species#18, catch_date#1644, length_cm#355

# INNER JOIN KE DATASET 1 DAN 4

In [ ]:
# 8. Lakukan inner join antara joined_df dan df_lobster_market_sales_dataset (Corrected based on 'on' column)
joined_df = joined_df.join(df_lobster_market_sales_dataset, on=["lobster_id"], how="inner")

joined_df.show()

+----------+-----------------+---------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+-------------------+--------+-----------------+------------+-----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+----------+--------------------+---------+----------+------------------+-----------------+---------------------+------------------------+----------------+--------------+----------------------+-----------+----------------+--------------------+---------+----------+-------------------+---------------+----------------+------------+--------------+------------------+-------------+-----------------+---------------+------------------+-------------------+
|lobster_id|fishing_ground_id|fisher_id|         species|catch_date|length_cm|weight_grams|carapace_length_

In [ ]:
from pyspark.sql.functions import year, col

# Extract 'sale_year' from the 'sale_date' column
# 'sale_date' is already a DateType according to the schema, so no need to convert with to_date
joined_df = joined_df.withColumn('sale_year', year(col('sale_date')))

# For seeing the results
joined_df.select("sale_date", "sale_year").show(5)
joined_df.printSchema()

+----------+---------+
| sale_date|sale_year|
+----------+---------+
|2024-10-29|     2024|
|2024-10-18|     2024|
|2024-09-12|     2024|
|2025-03-02|     2025|
|2025-05-29|     2025|
+----------+---------+
only showing top 5 rows

root
 |-- lobster_id: string (nullable = true)
 |-- fishing_ground_id: string (nullable = true)
 |-- fisher_id: string (nullable = true)
 |-- species: string (nullable = true)
 |-- catch_date: date (nullable = true)
 |-- length_cm: double (nullable = false)
 |-- weight_grams: double (nullable = false)
 |-- carapace_length_cm: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- shell_condition: string (nullable = false)
 |-- catch_method: string (nullable = true)
 |-- vessel_id: string (nullable = true)
 |-- market_price_per_kg: double (nullable = true)
 |-- is_alive: boolean (nullable = true)
 |-- inspection_passed: boolean (nullable = true)
 |-- export_ready: boolean (nullable = true)
 |-- fisher_name: string (nullable = true)
 |-- license_nu

In [ ]:
joined_df.show()

+----------+-----------------+---------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+-------------------+--------+-----------------+------------+-----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+----------+--------------------+---------+----------+------------------+-----------------+---------------------+------------------------+----------------+--------------+----------------------+-----------+----------------+--------------------+---------+----------+-------------------+---------------+----------------+------------+--------------+------------------+-------------+-----------------+---------------+------------------+-------------------+---------+
|lobster_id|fishing_ground_id|fisher_id|         species|catch_date|length_cm|weight_grams|carapa

6. **Mengitung Tren Nilai Penjualan Total berdasarkan Spesies Lobster (2024-2025)**

In [ ]:
# Syntax PySpark untuk groupby, sum, dan sort
from pyspark.sql.functions import sum as spark_sum

sales_date_year = joined_df.groupBy(['species', 'sale_year']) \
    .agg(spark_sum('total_sale_value').alias('total_sale_value')) \
    .orderBy('sale_year')
sales_date_year.show()

+----------------+---------+------------------+
|         species|sale_year|  total_sale_value|
+----------------+---------+------------------+
|    Rock Lobster|     2024|49013.945699999975|
| Slipper Lobster|     2024|48478.391299999974|
|European Lobster|     2024| 49564.14919999997|
|   Spiny Lobster|     2024| 46104.57119999994|
|American Lobster|     2024|45929.464899999984|
|   Spiny Lobster|     2025|32235.837799999976|
|American Lobster|     2025|        34756.7866|
|    Rock Lobster|     2025| 34046.98710000002|
| Slipper Lobster|     2025| 33623.10110000001|
|European Lobster|     2025| 34308.90299999997|
+----------------+---------+------------------+



**7.Menghitung Kuantitas Terjual berdasarkan Spesies (2024 vs 2025)'**

---



In [ ]:
from pyspark.sql.functions import sum

# Pastikan 'sale_year' sudah ada dan tipenya benar (int atau string)
# Jika 'sale_year' belum ada, Anda mungkin perlu membuatnya terlebih dahulu
# Contoh jika 'sale_date' ada dan Anda ingin mengekstrak tahunnya:
# joined_df3 = joined_df3.withColumn('sale_year', year(joined_df3['sale_date']))

quantity = joined_df.groupBy('species', 'sale_year').agg(sum('quantity_sold_kg').alias('total_quantity_sold_kg'))

# Untuk melihat hasilnya
quantity.show()
quantity.printSchema()

+----------------+---------+----------------------+
|         species|sale_year|total_quantity_sold_kg|
+----------------+---------+----------------------+
|   Spiny Lobster|     2025|     541.7800000000002|
|American Lobster|     2025|     563.0800000000006|
|    Rock Lobster|     2025|     569.7000000000005|
|    Rock Lobster|     2024|     805.0899999999998|
| Slipper Lobster|     2024|     808.5300000000009|
|European Lobster|     2024|     822.6200000000013|
| Slipper Lobster|     2025|     576.9400000000005|
|   Spiny Lobster|     2024|     774.0500000000001|
|American Lobster|     2024|     760.0700000000005|
|European Lobster|     2025|     567.5699999999994|
+----------------+---------+----------------------+

root
 |-- species: string (nullable = true)
 |-- sale_year: integer (nullable = true)
 |-- total_quantity_sold_kg: double (nullable = true)



**8.  Menghitung Rata-rata harga per kg berdasarkan Spesies (2024 vs 2025)**

In [ ]:
from pyspark.sql.functions import percentile_approx

# Hitung median price_per_kg per species dan sale_year
price = joined_df.groupBy('species', 'sale_year') \
                  .agg(percentile_approx('price_per_kg', 0.5).alias('median_price_per_kg'))

# Untuk melihat hasilnya
price.show()
price.printSchema()

+----------------+---------+-------------------+
|         species|sale_year|median_price_per_kg|
+----------------+---------+-------------------+
|   Spiny Lobster|     2025|               58.3|
|American Lobster|     2025|              61.87|
|    Rock Lobster|     2025|               59.4|
|    Rock Lobster|     2024|              60.44|
| Slipper Lobster|     2024|               59.8|
|European Lobster|     2024|              59.57|
| Slipper Lobster|     2025|              58.61|
|   Spiny Lobster|     2024|              59.66|
|American Lobster|     2024|              61.81|
|European Lobster|     2025|              61.54|
+----------------+---------+-------------------+

root
 |-- species: string (nullable = true)
 |-- sale_year: integer (nullable = true)
 |-- median_price_per_kg: double (nullable = true)



# INNER JOIN DATASET KE 1 DAN 5

In [ ]:
joined_df = joined_df.join(lobster_quality_inspections, on=["lobster_id"], how="inner")

joined_df.show()

+----------+-----------------+---------+----------------+----------+---------+------------+------------------+-------+---------------+------------+---------+-------------------+--------+-----------------+------------+----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+----------+--------------------+---------+----------+------------------+-----------------+---------------------+------------------------+----------------+--------------+----------------------+-----------+----------------+--------------------+---------+----------+-------------------+---------------+----------------+------------+--------------+------------------+-------------+-------------------+---------------+------------------+--------------------+---------+-------------+---------------+--------------+-----------------+---------------+---------------+-

**9. menghitung Total Tangkapan per Spesies berdasarkan Kualitas Daging (dengan Nilai)**

In [ ]:
from pyspark.sql.functions import sum

pivot_table4 = joined_df.groupBy("species") \
    .pivot("meat_quality") \
    .agg(sum("total_catch_this_year"))

pivot_table4.show()


+----------------+--------------+-------+--------+
|         species|Below Standard|Premium|Standard|
+----------------+--------------+-------+--------+
|American Lobster|        110501|  93419|  107517|
|    Rock Lobster|         60457|  99629|  114143|
| Slipper Lobster|        105069| 126221|  130468|
|   Spiny Lobster|         76903|  45089|  101580|
|European Lobster|         89363| 113561|  109086|
+----------------+--------------+-------+--------+



# INNER JOIN DATASET KE 1 DAN 6

In [ ]:
# inner join dataset 1-6 (now including vessel data)
# Join with lobster_vessels_dataset using 'vessel_id' as the join key
joined_df = joined_df.join(lobster_vessels_dataset, on=["vessel_id"], how="inner")

joined_df.show()

+---------+----------+-----------------+---------+----------------+----------+---------+------------+------------------+-------+---------------+------------+-------------------+--------+-----------------+------------+----------------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+----------------+--------------+------------+---------+---------------+---------------------+----------+--------------------+---------+----------+------------------+-----------------+---------------------+------------------------+----------------+--------------+----------------------+-----------+----------------+--------------------+---------+----------+-------------------+---------------+----------------+------------+--------------+------------------+-------------+-------------------+---------------+------------------+--------------------+---------+-------------+---------------+--------------+-----------------+---------------+---------------+-

**10. menghitung Rincian Total Tangkapan per Spesies menurut Jenis Kapal (Bertumpuk)**

In [ ]:
from pyspark.sql.functions import sum

aa = joined_df.groupBy("species") \
    .pivot("vessel_type") \
    .agg(sum("total_catch_this_year"))

aa.show()


+----------------+--------+------------+---------------+-------+
|         species|Day Boat|Lobster Boat|Offshore Vessel|Trawler|
+----------------+--------+------------+---------------+-------+
|American Lobster|   97567|       84934|          78221|  50715|
|    Rock Lobster|   74612|       64792|          71562|  63263|
| Slipper Lobster|  117996|       77056|          81603|  85103|
|   Spiny Lobster|   52594|       43262|          61819|  65897|
|European Lobster|   78668|       77282|          88954|  67106|
+----------------+--------+------------+---------------+-------+

